# What Day of the Week Is It? Three Methods.

## Julian Day

I first learned a method for determining the day of the week for any given day from the book *Astronomical Algorithms* by Jean Meeus. I have the second edition in front of me now, and the instructions are given as a kind of corollary to the main topic of Chapter 7 on computing the Julian Day Number for a given calendar day. Julian Day Number is somewhat confusingly named, since is doesn’t mean, “A day in the Julian Calendar,” but rather, “the count of days (and fractions thereof) since Greenwich noon on the beginning of the year -4712.” My implementation of the algorithm is:

In [1]:
function JulianDay(Y, M, D)
    isGregorian = (Y > 1582) || (Y == 1582 && M > 10) || (Y == 1752 && M == 10 && D > 4)
    
    if M < 3
        Y -= 1
        M += 12
    end
    
    B = 0
    if isGregorian
        A = Y ÷ 100
        B = 2 - A + A ÷ 4
    end
    
    floor(365.25 * (Y + 4716)) + floor(30.6001 * (M + 1)) + D + B - 1524.5    
end

JulianDay (generic function with 1 method)

As an example, the Julian Day number for the start of our first Number Theory class on January 14 at 21:40 UTC is:

In [2]:
using Printf
@printf("%.1f", JulianDay(2020, 1, 14.9))

2458863.4

### Some discussion of the code

The `Y`, `M`, and `D` function parameters correspond to the full year, the month number (1 for January, etc), and the (fractional) day of the month. Fun fact: I learned during the course of my research that the mathematician Pierre-Simon Laplace was the first to express a day and time together in this way. So, for example, January 14, 2020 at 21:40 corresponds to `Y = 2020`, `M = 1`, and `D = 14.9` where $14.9 = 14 + 21 \div 24 + 40 \div (24 \times 60)$.

The first line determines if the given date is in the Gregorian or Julian calendar. While the political implementation details are complicated, for our purposes, Gregorian calendar reform means that, in 1582, Thursday, October 4 was followed by Friday, October 15. Fortunately, for the task of determining the day of the week, no discontinuity was introduced here.

The next section addresses the fact that leap days are added to the end of February (month 2). If we pretend that years start at the beginning of March (as it actually did for many ancient peoples), then the task of figuring how many days have elapsed in a year based on the current month is made easier.

The next section computes a correction factor for the more complicated leap year rules introduced by Gregorian calendar reform. Whereas the Julian calendar adds a leap day every fourth year (which is why you’ll see the figure $365.25$ as the length of a year), the Gregorian calendar skips leap days in century years [$Y \equiv 0 \pmod{100}$] but adds them every fourth century year [$Y \equiv 0 \pmod{400}$]. Also, remember that `÷` is integer division in Julia.

The last line returns the Julian Day. The first term is (roughly) the number of days from day zero (offset by four years to give some wiggle room when computing days near day 0 — the method is not meant to be used for dates that would result in a negative JD number) to the beginning of year Y. The next term is the number of days from the beginning of year Y to the beginning of month M. This is very interesting and I’ll discuss it more below. The last terms are the number of days since the beginning of this month, the Gregorian leap day correction, and correction for all the other offsets we’ve made.

Our months are mess from a computational standpoint. Months 1, 3, 5, 7, 8, 10, & 12 have 31 days. Months 4, 6, 9, & 11 have 30. Month 2 has 28 or 29 days. Another advantage to pretending that the year starts in March is that it starts a repeating pattern of month lengths:

<table>
    <tbody>
        <tr><td>Mar</td><td>Aug</td><td>Jan</td><th>31</th></tr>
        <tr><td>Apr</td><td>Sep</td><td>   </td><th>30</th></tr>
        <tr><td>May</td><td>Oct</td><td>   </td><th>31</th></tr>
        <tr><td>Jun</td><td>Nov</td><td>   </td><th>30</th></tr>
        <tr><td>Jul</td><td>Dec</td><td>   </td><th>31</th></tr>
    </tbody>
</table>

No, February isn’t on that that chart, but as the last month of the year, we don’t need it, we just need the number of the day in February. Also, what we really want is the cumulative number of days in the given “year” before the start of the given month:

<table>
    <tbody>
        <tr><td>3</td><td>Mar</td><th>0</th><td></td></tr>
        <tr><td>4</td><td>Apr</td><th>31</th><td>+31</td></tr>
        <tr><td>5</td><td>May</td><th>61</th><td>+30</td></tr>
        <tr><td>6</td><td>Jun</td><th>92</th><td>+31</td></tr>
        <tr><td>7</td><td>Jul</td><th>122</th><td>+30</td></tr>
        <tr><td>8</td><td>Aug</td><th>153</th><td>+31</td></tr>
        <tr><td>9</td><td>Sep</td><th>184</th><td>+31</td></tr>
        <tr><td>10</td><td>Oct</td><th>214</th><td>+30</td></tr>
        <tr><td>11</td><td>Nov</td><th>245</th><td>+31</td></tr>
        <tr><td>12</td><td>Dec</td><th>275</th><td>+30</td></tr>
        <tr><td>1</td><td>Jan</td><th>306</th><td>+31</td></tr>
        <tr><td>2</td><td>Feb</td><th>337</th><td>+31</td></tr>
    </tbody>
</table>

If we look at the values the `floor(30.6001 * (M + 1))` term takes and subtract off an offset, we can see the same results:

In [3]:
html = "<table><thead><tr><th>m</th><th>M</th><th>term</th><th>cumulative</th><th>days</th></tr></thead><tbody>"
for m in [3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 1, 2]
    M = m
    if M < 3
        M += 12
    end
    c = Integer(floor(30.6001 * (M+1)))
    d = Integer(floor(30.6001 * (M+1))) - 122
    e = Integer(floor(30.6001 * (M+1)) - floor(30.6001 * (M)))
    f = (m == 3 ? "" : "+$e")
    html *= "<tr><th>$m</th><td>$M</td><td>$c</td><td>$d</td><td>$f</td></tr>"
end
html *= "</tbody></table>"
IJulia.display("text/html", html)

m,M,term,cumulative,days
3,3,122,0,
4,4,153,31,+31
5,5,183,61,+30
6,6,214,92,+31
7,7,244,122,+30
8,8,275,153,+31
9,9,306,184,+31
10,10,336,214,+30
11,11,367,245,+31
12,12,397,275,+30


Observe that the number in the days column is the number of days in the previous month. These get added to the cumulative total. Note, the code uses 30.6001 instead of 30.6 to account for the vagaries of floating point implementation/representation details.

### Day of the Week
Having found the Julian Day number, it is then a matter of knowing what day of the week JD<sub>0</sub> was and comparing our day, modulo 7.

In [4]:
function DayOfWeek(Y, M, D)
    ["Sunday", "Monday", "Tuesday", "Wednesday", "Thursday", "Friday", "Saturday"][1+mod(Integer(1.5 + JulianDay(Y, M, floor(D))), 7)]
end


DayOfWeek (generic function with 1 method)

Looking again at the day of our first Number Theory class we see that it was on:

In [5]:
DayOfWeek(2020, 1, 14)

"Tuesday"

## The Doomsday Algorithm

John Conway devised this method with inspiration from Martin Gardner and Lewis Carroll. It’s intended to be something that can be performed purely mentally, so it keeps computations simpler at the expense of more memorization. I’ll implement it in code since I won’t be around to do it on command.

In [6]:
function DoomsdayAlgorithm(year, month, date)
    weekday = Dict(0 => "Sunday", 1 => "Monday", 2 => "Tuesday", 3 => "Wednesday", 4 => "Thursday", 5 => "Friday", 6 => "Saturday")
    centuryDay = Dict(16 => 2, 17 => 0, 18 => 5, 19 => 3, 20 => 2, 21 => 0)
    doomsday = Dict(3 => 0, 4 => 4, 5 => 9, 6 => 6, 7 => 11, 8 => 8, 9 => 5, 10 => 10, 11 => 7, 12 => 12)
    isLeapYear = mod(year, 4) == 0 && (mod(year, 100) != 0 || mod(year, 400) == 0)
    doomsday[1] = isLeapYear ? 4 : 3 # the doomsdays for January and February differ in leap years
    doomsday[2] = isLeapYear ? 29 : 28
    cc, yy = divrem(year, 100) # 1971 => 19, 71

    thumb = mod(date - doomsday[month], 7) # Doomsday Difference
    index = centuryDay[cc] # Century Day
    middle = div(yy, 12) # Number of Dozens
    ring = rem(yy, 12) # Remainder
    pinkie = div(ring, 4) # Leap Years in Remainder
    
    weekday[mod(thumb + index + middle + ring + pinkie, 7)]
end

    

DoomsdayAlgorithm (generic function with 1 method)

The general idea is that there are dates that all fall on the same day of the week in a given year. If you can figure out what day that is for the year of interest, then you can do a manageable difference, modulo 7 between the known day and the day of interest to find the day of the week. For our example of January 14, 2020, we start by observing that January 4 falls on doomsday and the 14th is $14 - 4 \pmod{7} = 3$ weekdays days later. The century day for 2020 is Tuesday (2). Then there are adjustment terms. We add them all up, modulo seven and translate the number into a weekday.

In [7]:
DoomsdayAlgorithm(2020, 1, 14)

"Tuesday"

## Lewis Carroll’s Method

This method was published as a [short article](https://www.nature.com/articles/035517a0) in *Nature* in 1887. Here’s my implementation:

In [8]:
function DayOfTheWeek(year, month, date)
    isNewStyle = (year > 1752) || (year == 1752 && month > 9) || (year == 1752 && month == 9 && date > 2)
    isLeapYear = mod(year, 4) == 0 && (mod(year, 100) != 0 || mod(year, 400) == 0)

    cc, yy = divrem(year, 100) # 1971 => 19, 71
    centuryItem = isNewStyle ? 2 * (3 - rem(cc, 4)) : 18 - cc
#     println("centuryItem: $centuryItem")
    dozens, overplus = divrem(yy, 12)
    yearItem = dozens + overplus + div(overplus, 4)
#     println("yearItem: $yearItem")
    forMonth = Dict(
         1 => 0,  # given
         2 => 3,  # given
         3 => 3,  # given
         4 => 6,  # x 31 days in March (no vowel) gives 3. 3 + 31 gives 6: right
         5 => 1,  # 10 - 4 + 30 · vowel (April)
         6 => 4,  # x if y is a vowel, then 10 - 5 + 31 (days in May) gives 1. 5 + 31 gives 1: wrong
         7 => 6,  # 10 - 6 + 30 · vowel (June)
         8 => 2,  # x if y is a vowel, then 10 - 7 + 31 (days in July) gives 6. 7 + 31 gives 3: wrong
         9 => 5,  # 10 - 8 + 31 · vowel (August)
        10 => 0,  # x 30 days in September (no vowel) gives 2. 9 + 30 gives 4: wrong
        11 => 3,  # 10 - 10 + 31 · vowel (October)
        12 => 5   # 12 given
    )
    monthItem = forMonth[month]
#     println("monthItem: $monthItem")
    dayItem = date
#     println("dayItem: $dayItem")
    total = centuryItem + yearItem + monthItem + dayItem
    if isLeapYear && month < 3
        total -= 1
    end
#     println("total: $total")
    ["Sunday", "Monday", "Tuesday", "Wednesday", "Thursday", "Friday", "Saturday"][1 + mod(total, 7)]
end

DayOfTheWeek (generic function with 1 method)

Carroll lived in the British Empire, which didn’t adopt Gregorian calendar reform until September, 1752, hence the rules for his “old style” and “new style” dates. I haven’t been able to find an interpretation for his *Month-Item* term that works for all months: I’ve had to fill them in empirically. Carroll’s instructions:

> *The Month-Item*.—If it begins or ends with a vowel,
> subtract the number, denoting its place in the year, from
> 10. This, plus its number of days, gives the item for the
> following month. The item for January is “0”; for
> February or March (the 3rd month), “3”; for December
> (the 12th month), “12.”

don’t explain what to do with the months that *don’t* begin or end with a vowel, or give any indication of whether Carroll considers the *y* at the end of May & July vowels—probably not, though, following the vowel rule for May & July gives bad results. The two examples given in the article don’t use non-vowel months, unfortunately. Trying to follow old math reminds me of trying to cook from old recipies. I’m grateful to the writer who pointed out that for Carroll’s “overplus” I should read, “remainder”.

Once I determined correct values for the `monthItem`, the procedure does work:

In [9]:
[(m, DayOfWeek(2020, m, 14), DayOfTheWeek(2020, m, 14)) for m in 1:12]

12-element Array{Tuple{Int64,String,String},1}:
 (1, "Tuesday", "Tuesday")     
 (2, "Friday", "Friday")       
 (3, "Saturday", "Saturday")   
 (4, "Tuesday", "Tuesday")     
 (5, "Thursday", "Thursday")   
 (6, "Sunday", "Sunday")       
 (7, "Tuesday", "Tuesday")     
 (8, "Friday", "Friday")       
 (9, "Monday", "Monday")       
 (10, "Wednesday", "Wednesday")
 (11, "Saturday", "Saturday")  
 (12, "Monday", "Monday")      

---
— Rob Ireton · April 26, 2020 (a Sunday!)